In [2]:
from tika import parser # pip install tika

raw_xml = parser.from_file('ml_book.pdf', xmlContent=True)
body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
body_without_tag = body.replace("<p>", "").replace("</p>", "").replace("<div>", "").replace("</div>","").replace("<p />","")
text_pages = body_without_tag.split("""<div class="page">""")[1:]

In [258]:
len(text_pages)

330

In [3]:
text_pages = [x.replace('-\n ','').replace('-\n', '') for x in text_pages]

In [118]:
text_pages[89]

'\nID Bread Cheese Wine Milk Fruit\nt1 T T F F T\nt2 T F F T F\nt3 T T T F T\nt4 F F F T T\nt5 F T T F F\n\nAn example of an itemset is {bread, cheese}. This itemset is contained in transactions t1 and t3.\n\nAn association rule is of the form X → Y (also written as if X then Y ), with\nX and Y itemsets. In the context of market basket analysis, the rule is usually\ninterpreted as saying “people who buy X usually also buy Y ”. More generally, we\ncan interpret it as “transactions that contain X usually also contain Y ”.\n\nThis may seem similar to the interpretation of classification rules, but note the\nfollowing differences:\n\n1. Y is a set of attributes, not a single attribute;\n\n2. In classification rules the single attribute Y is predefined: we know what we\nneed to predict. In association rules, on the other hand, Y may contain any\nattributes, and it is part of the learning task to find an interesting set of\nattributes Y ;\n\n3. In “transactions that contain X usually also co

In [4]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
whole = '\n'.join(text_pages)

In [8]:
doc = nlp(whole)

In [231]:
terms,inds = list(map(list, zip(*[(doc[tok.i: tok.head.i + 1],list(range(tok.i, tok.head.i + 1)))  for tok in doc if (tok.dep_ == 'compound') and (all(len(x)>2 for x in doc[tok.i: tok.head.i + 1].text.strip().split()))])))

In [234]:
inds = [x for y in inds for x in y]

In [235]:
terms = terms + [ tok  for tok in doc if tok.dep_ in ('nsubj', 'acomp') and not tok.is_stop and not tok.i in inds]

In [236]:
terms = list(set(x.text.lower() for x in terms))

In [237]:
terms = [t for t in terms if all(x.isalpha() or x in ('.',' ') for x in t)]

In [238]:
terms = [t for t in terms if all(len(x) > 1 for x in t.split())]

In [241]:
[t for t in terms if len(t.split())]

['output error',
 'info function',
 'step',
 'general',
 'carlo policy iteration',
 'exploration strategy',
 'parameter settings',
 'definition',
 'clauses',
 'related',
 'set format',
 'learning method',
 'horn clause',
 'properness',
 'machine',
 'di',
 'logic formulas',
 'availability',
 'terminal',
 'finite hypothesis space',
 'burglary',
 'satisfied',
 'fourteenth international joint conference',
 'nodes',
 'wj',
 'regression models',
 'model chunk',
 'clustering system',
 'construction process',
 'contour lines',
 'string',
 'bayes learning',
 'reason',
 'optimal split',
 'policy algorithm',
 'birds',
 'years',
 'cluster seeds',
 'prediction task',
 'areas',
 'consistent',
 'disjoint',
 'representation power',
 'descriptive',
 'transformation',
 'ranking',
 'board configurations',
 'modification',
 'news',
 'unstable',
 'mining toolbox',
 'close',
 'output perceptrons',
 'pruning algorithm',
 'problem statements',
 'type',
 'clustering systems',
 'tree automata',
 'auai press',
 

In [242]:
terms = [t for t in terms if len(t.split()) < 5]

In [243]:
terms = sorted(terms)[::-1]

In [244]:
len(terms)

2572

In [245]:
vocab = terms

In [246]:
tokenizer = spacy.tokenizer.Tokenizer(spacy.vocab.Vocab(strings=vocab))

In [247]:
set([len(x.split()) for x in terms])

{1, 2, 3, 4}

In [248]:
terms

['ρl',
 'δc',
 'µi',
 'zero',
 'zag shape',
 'yk',
 'yj',
 'yij',
 'yi',
 'years',
 'year',
 'xri',
 'xor function',
 'xml tree',
 'xij',
 'xi',
 'xd',
 'xca',
 'xac',
 'wrong',
 'worse',
 'world states',
 'work',
 'word',
 'won games',
 'wj',
 'wise',
 'winner',
 'wiley new york',
 'wikipedia entry',
 'width',
 'wider',
 'wide',
 'wi',
 'white',
 'whichever',
 'wettelijk depot',
 'weka data mining tool',
 'weights',
 'weighting instances',
 'weighting attributes',
 'weighted',
 'weight wji',
 'weight vectors',
 'weight vector',
 'weight updates',
 'weight settings',
 'weight matrix',
 'weight matrices',
 'weight',
 'web pages',
 'web page',
 'web',
 'weather',
 'weaker',
 'weak',
 'ways',
 'way',
 'warmr system',
 'vultures',
 'voting procedure',
 'voting',
 'voronoi diagram',
 'voorafgaande schriftelijke',
 'voor aanvragen tot',
 'vocabulary',
 'vladimir vapnik',
 'visualisation',
 'visible',
 'vice versa',
 'vi',
 'versionspaces approach',
 'versions space',
 'version spaces',
 'ver

In [345]:
!pip install inflect

In [380]:
import inflect

engine = inflect.engine()

In [381]:
t = terms[0]

In [386]:
def to_singular(x):
    if (len(x) < 3) or x in ('bayes',) or x.endswith('as') or x.endswith('ss') or x.endswith('us') or x.endswith('is'):
        return x
    if x == 'matrices':
        return 'matrix'
    ret = engine.singular_noun(x)
    if ret:
        return ret
    return x

In [387]:
singular_terms = list(set([' '.join([to_singular(x)  for x in t.split()]) for t in terms]))

In [388]:
import re
counts = np.zeros((len(text_pages), len(singular_terms)))
for pcnt, p in enumerate(text_pages):
    p = ' '.join([''.join([y for y in x if y.isalpha() or y == '.']) for x in p.split()]).lower()
    findings = {}
    for t in terms:
        f = [range(m.start() + 1,m.end()) for m in re.finditer(f' {t} ', p)]
        if f:
            findings[t] = f
    are_ngrams = [x for x in findings if len(x.split())>1]
    ngrams_findings_inds = [x for k in are_ngrams for y in findings[k] for x in y]
    findings = {k:[x for x in t if k in are_ngrams or all(y not in ngrams_findings_inds for y in x)] for k,t in findings.items()}
    findings = {k:t for k,t in findings.items() if k}
    try:
        finds, fcounts =  list(map(list, zip(*[[singular_terms.index(' '.join([to_singular(x) for x in k.split()])),len(t)] for k, t in findings.items()])))
    except:
        print(p)
        continue
    counts[pcnt, finds] = counts[pcnt, finds] + fcounts


textbook on bayesian networks. 


In [389]:
counts.sum(axis=0)

array([ 2., 33.,  3., ...,  1., 20.,  1.])

In [390]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit_transform(counts)

In [391]:
tfidf = tfidf.toarray()

In [392]:
import pandas as pd
terms_df = pd.DataFrame(tfidf, columns=singular_terms)

In [393]:
page_offset = 1

In [394]:
keep = 10
start_from_page = 11

In [395]:
index_page_series = terms_df.aggregate(lambda x: x[x>0].index[np.argsort(-x[x>0])] - page_offset)
index_page_series = index_page_series.apply(lambda x: x[x>start_from_page][:keep])
index_page_series = index_page_series.apply(lambda x: ','.join(str(y) for y in x))


In [396]:
index_page_series = index_page_series[index_page_series != '']

In [397]:
index_page_series = index_page_series[index_page_series != '0']

In [401]:
index_page_series.sort_index().to_excel('output.xlsx',header=False)